Run #1 in order of benchmarking

This file processes the existing processed datafile, adds laggs and filters for 2021, and then removes some columns, then returns the file in the benchmark folder. This file is later used by the BS file.

In [9]:
#TODO : Unable to import these functions in parent directory

def lag_features(df, features, seq_length):
    """Transforms a raw 2D dataframe of option data into 2D dataframe ofsequence data.
    Last 2 indexes per sequence are bid and ask price. The len(features)*seq_length
    features before are sequences of features"""
    df = df.sort_values(["Expire_date", "Strike", "Ttl"], ascending = [True, True, False])
    
    # Adding lag for naive benchmarking
    df["Naive"] = df["Price"].shift(1)

    for step in range(seq_length)[::-1]:
        for feature in features:
            df[feature + "-" + str(step)] = df[feature].shift(step)
    
    df["Check_strike"] = df["Strike"] == df["Strike"].shift(seq_length-1)
    df["Check_expire"] = df["Expire_date"] == df["Expire_date"].shift(seq_length-1)
    df = df[(df["Check_strike"] == True) & (df["Check_expire"] == True)]
    df = df.drop(["Check_strike", "Check_expire"], axis=1)
    #df[["Bid_strike_last", "Ask_strike_last"]] = df[["Bid_strike", "Ask_strike"]]
    #df[["Bid_last", "Ask_last"]] = df[["Bid", "Ask"]]
    df["Price_last"] = df["Price"]
    df = df.sort_values(["Quote_date"], ascending = [True])
    return df

    
def read_file(file):
    """Read a single file and return a dataframe"""
    return pd.read_csv(file, skipinitialspace=True)



from math import log, sqrt, pi, exp
from scipy.stats import norm
import pandas as pd
import tensorflow as tf

In [10]:
file = f"../data/predictions/2021_predictions_11-21_12-51.csv"
df_options = read_file(file)

In [11]:
features = ["Underlying_last", "Strike", "Ttl", "Volatility", "R"]
seq_length = 5
num_features = len(features)
num_outputs = 1

df_options = lag_features(df_options, features, seq_length)

In [12]:
df_options = df_options[["Quote_date", "Underlying_last", "Strike", "Ttl", "Volatility", "R", "Price", "Naive", "Prediction"]]
df_options['Quote_date'] = pd.to_datetime(df_options.Quote_date, format='%Y-%m-%d')
df_options = df_options[df_options.Quote_date.dt.year == 2021]

# To be used for testing on for faster computing
#df_options_short = df_options[df_options["Quote_date"] == "2021-01-06"]

In [13]:
print(df_options.head())

      Quote_date  Underlying_last  Strike  Ttl  Volatility     R     Price  \
1429  2021-01-08          3824.45  1000.0    3    0.186024  0.08  2826.250   
79288 2021-01-08          3824.45  3860.0   49    0.186024  0.08    75.200   
79325 2021-01-08          3824.45  3865.0   49    0.186024  0.08    72.955   
79362 2021-01-08          3824.45  3870.0   49    0.186024  0.08    70.550   
79399 2021-01-08          3824.45  3875.0   49    0.186024  0.08    67.495   

          Naive   Prediction  
1429   2803.855  2774.402600  
79288    67.900    69.269806  
79325    65.210    66.548515  
79362    63.055    63.835953  
79399    60.500    61.140602  


In [14]:
mse_bs = tf.keras.metrics.mean_squared_error(
    df_options["Price"], df_options["Naive"]
)

In [15]:
print("MSE: ", mse_bs)
print("RMSE: ", sqrt(mse_bs))

MSE:  tf.Tensor(784.3698665832319, shape=(), dtype=float64)
RMSE:  28.006603981618905


In [17]:
# Write to file
df_options.to_csv('data/benchmark_with_pred_naive.csv', encoding='utf-8', index=False)

NameError: name 'df' is not defined